# Modifica CSV
Alcuni metodi per modificare velocemente i dati in csv. Sotto l'intero codice esportato in un file .py

In [2]:
import pandas as pd
import time
from datetime import datetime

In [4]:
#cOSTANTI
FILE_MONGO = "..\mongo.csv"

In [5]:
def rimuovi_NaN(data):
    data = data[pd.notnull(data['objectId'])]
    data = data[data['objectId']!="null"]
    return data

def col_time(data):
    timestamp = data['timestamp']
    datacorrect = []
    i = 0
    while(i<len(timestamp)):
        d = str(datetime.fromtimestamp(timestamp[i]/1e3))
        datacorrect.append(d[0:len(d)-7])
        i += 1
    data['time'] = datacorrect
    return data

def col_eventNumber(data):
    events = data['eventName']
    eventNumber = []
    i=0
    while(i<len(events)):
        if(events[i]=="heating"):
            eventNumber.append(700)
        if(events[i]=="coffee"):
            eventNumber.append(1500)
        if(events[i]=="standby"):
            eventNumber.append(200)
        if(events[i]=="off"):
            eventNumber.append(0)
        if(events[i]=="on"):
            eventNumber.append(500)
        i += 1
    data['eventNumber'] = eventNumber
    return data

def crea_csv_nuovo(data,name):
    data.to_csv(name)
    
def modifica_ore(data,ore,ms):
    d = str(data['time'][0])
    d = d[0:len(d)-ms]
    time.mktime(datetime.strptime(d, "%Y-%I-%d %H:%M:%S").timetuple())
    d=datetime.fromtimestamp(time.mktime(datetime.strptime(d, "%Y-%I-%d %H:%M:%S").timetuple())+(60*60*2))
    data['time']=d
    return data

In [6]:
mongo = pd.read_csv(FILE_MONGO)
mongo = rimuovi_NaN(mongo)
print(mongo.head())
nespresso   = mongo[mongo['objectId']=="nespresso_jolmilano"].reset_index(drop=True)
print(nespresso.head(3))
groundtruth = mongo[mongo['objectId']=="nespresso_jolmilano_groundtruth"].reset_index(drop=True)
print(groundtruth.head(3))

                 objectId                          time     timestamp  \
6094  nespresso_jolmilano  2018-06-27T17:38:48.560+0200  1.530114e+12   
6095  nespresso_jolmilano  2018-06-27T17:39:03.938+0200  1.530114e+12   
6096  nespresso_jolmilano  2018-06-27T17:39:25.601+0200  1.530114e+12   
6097  nespresso_jolmilano  2018-06-27T17:39:28.587+0200  1.530114e+12   
6098  nespresso_jolmilano  2018-06-27T17:39:37.840+0200  1.530114e+12   

     eventName  
6094   heating  
6095    coffee  
6096   standby  
6097    coffee  
6098   standby  
              objectId                          time     timestamp eventName
0  nespresso_jolmilano  2018-06-27T17:38:48.560+0200  1.530114e+12   heating
1  nespresso_jolmilano  2018-06-27T17:39:03.938+0200  1.530114e+12    coffee
2  nespresso_jolmilano  2018-06-27T17:39:25.601+0200  1.530114e+12   standby
                          objectId               time     timestamp eventName
0  nespresso_jolmilano_groundtruth  2018-7-2T11:30:30  1.530524e+12    co

In [7]:
nespresso = col_time(nespresso)
nespresso = col_eventNumber(nespresso)
nespresso.head()

,objectId,time,timestamp,eventName,eventNumber
0,nespresso_jolmilano,2018-06-27 17:38:48,1.530114e+12,heating,700
1,nespresso_jolmilano,2018-06-27 17:39:03,1.530114e+12,coffee,1500
2,nespresso_jolmilano,2018-06-27 17:39:25,1.530114e+12,standby,200
3,nespresso_jolmilano,2018-06-27 17:39:28,1.530114e+12,coffee,1500
4,nespresso_jolmilano,2018-06-27 17:39:37,1.530114e+12,standby,200


In [8]:
groundtruth = col_time(groundtruth)
groundtruth = col_eventNumber(groundtruth)
groundtruth.head()

,objectId,time,timestamp,eventName,eventNumber
0,nespresso_jolmilano_groundtruth,2018-07-02 11:30:30,1.530524e+12,coffee,1500
1,nespresso_jolmilano_groundtruth,2018-07-02 11:30:38,1.530524e+12,on,500
2,nespresso_jolmilano_groundtruth,2018-07-02 15:03:40,1.530537e+12,on,500
3,nespresso_jolmilano_groundtruth,2018-07-02 15:04:21,1.530537e+12,coffee,1500
4,nespresso_jolmilano_groundtruth,2018-07-02 17:31:52,1.530546e+12,on,500


In [9]:
nespresso.to_csv("manu_rules.csv")
groundtruth.to_csv("groudtruth.csv")

# File su potenza
Modifico il file di openhab che raccoglie dati sulla potenza

In [31]:
FILE_OPENHAB="../openhab_today_corrected.csv"

In [39]:
openhab = pd.read_csv(FILE_OPENHAB)
openhab['time'] = openhab['time_correct']
openhab.drop(axis=1,labels='time_correct',inplace=True)
openhab.to_csv("power.csv")
openhab.head()

,Unnamed: 0,time,value
0,0,2018-07-03 00:00:00,0.000000
1,1,2018-07-03 00:07:37,0.292534
2,2,2018-07-03 00:07:40,0.000000
3,3,2018-07-03 00:15:31,0.278355
4,4,2018-07-03 00:15:34,0.000000


# Creazione file .py 
Questo file sarà eseguibile da riga di comando con "python nomefile.py". (NB: si dovrà installare un compilatore python e la libreria pandas)

In [26]:
%%writefile modifica_csv.py

import pandas as pd
import time
from datetime import datetime

def rimuovi_NaN(data):
    #Elimina le righe contenenti NaN e null nella colonna 'objectId'
    #data: il dataframe che si vuole modificare
    data = data[pd.notnull(data['objectId'])]
    data = data[data['objectId']!="null"]
    return data

def col_time(data):
    #Legge la colonna "timestamp" e una nuova colonna con la data in formato
    #"%Y-%I-%d %H:%M:%S"
    #data: il dataframe che si vuole modificare
    timestamp = data['timestamp']
    datacorrect = []
    i = 0
    while(i<len(timestamp)):
        d = str(datetime.fromtimestamp(timestamp[i]/1e3))
        datacorrect.append(d[0:len(d)-7])
        i += 1
    data['time'] = datacorrect
    return data

def col_eventNumber(data):
    #Sostituisce i nomi degli eventi sulla colonna "eventName" con dei valori numerici
    #data: il dataframe che si vuole modificare
    events = data['eventName']
    eventNumber = []
    i=0
    while(i<len(events)):
        if(events[i]=="heating"):
            eventNumber.append(700)
        if(events[i]=="coffee"):
            eventNumber.append(1500)
        if(events[i]=="standby"):
            eventNumber.append(200)
        if(events[i]=="off"):
            eventNumber.append(0)
        if(events[i]=="on"):
            eventNumber.append(500)
        i += 1
    data['eventNumber'] = eventNumber
    return data

def crea_csv_nuovo(data,name):
    #Crea un nuovo file csv
    #data: il dataframe che si vuole modificare
    #name: nome del nuovo file
    data.to_csv(name+".csv")
    
def modifica_ore(data,ore,ms):
    #data: il dataframe che si vuole modificare
    #ore: quante ore si vogliono sommare alla data 
    #ms: numero di caratteri, nella data presente in tabella, presenti dopo i secondi
    #(esempio: per "2018-07-02 11:30:30.05" ms=3 per eliminare ".05")
    d = str(data['time'][0])
    d = d[0:len(d)-ms]
    time.mktime(datetime.strptime(d, "%Y-%I-%d %H:%M:%S").timetuple())
    d=datetime.fromtimestamp(time.mktime(datetime.strptime(d, "%Y-%I-%d %H:%M:%S").timetuple())+(60*60*2))
    data['time']=d
    return data

#Codice di esempio che usa i metodi sopra (senza creare nuovi file):
#cOSTANTI
FILE_MONGO = "mongo.csv"

mongo = pd.read_csv(FILE_MONGO)
mongo = rimuovi_NaN(mongo)
print(mongo.head())
nespresso   = mongo[mongo['objectId']=="nespresso_jolmilano"].reset_index(drop=True)
print(nespresso.head(3))
groundtruth = mongo[mongo['objectId']=="nespresso_jolmilano_groundtruth"].reset_index(drop=True)
print(groundtruth.head(3))

nespresso = col_time(nespresso)
nespresso = col_eventNumber(nespresso)
nespresso.head()

groundtruth = col_time(groundtruth)
groundtruth = col_eventNumber(groundtruth)
groundtruth.head()

Overwriting modifica_csv.py
